# Street Roughness Predictor for Bikes
### Written by: Michael Krebsbach, Braden Michelson, and Oren Erlich

# Imports
We make use of the following libraries and external code sources: 
- **Tensorflow**: the structure of the NN
- **numpy**: for data manipulation
- **matplotlib**: plotting the data and images
- The rest of the imports are misc for running the NN

Tutorials used: https://www.tensorflow.org/tutorials/load_data/images

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf

#allow access to gdrive
from google.colab import drive
import pathlib

#file manipulation
import os
import PIL
import PIL.Image
import tensorflow_datasets as tfds

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)


2.8.0


# Pre-Process Data

Create a shortcut of this Google Drive folder in your main Google drive directory (a.k.a "My Drive"). The resulting path should be `My Drive/Pictures`. If the directory is placed anywhere else you will need to adjust the `data_dir` string to point to the correct location.

This directory contains the pre-sorted training images of the asphalt at different roughness levels that will be fed into the neural network to generate the model.

https://drive.google.com/drive/folders/1sgRUGI_mXmF1Ue8zNnxHRU0tR5-QgBfU?usp=sharing

In [ ]:

drive.mount('/content/gdrive', force_remount=True)
data_dir = "gdrive/My Drive/Pictures"
data_dir = pathlib.Path(data_dir)


Mounted at /content/gdrive


In this step we simply get the number of image files recognized within Colab. The target number, assuming that the above step was done correctly, should be 99.

In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

99


We get the list of all .jpg files in the mounted directory, then render the first in the list to verify that it is not corrupted. 

The rendering code has been commented out as it is quite slow to run. Uncomment it out if you want to try it out!

In [ ]:
road = list(data_dir.glob('*/*.jpg'))
# PIL.Image.open(str(road[0]))


NameError: ignored

# Initialize the Neural Network

Here we set up the information for the initial data set. You may notice that the `img_height` and `img_width` variables do not match with the resolution of the input images you will find in the provided `/Pictures/` directory. This is due to both limits on free Google GPU capacity, as well as observed improvements to accuracy when images were compressed to this size.

We initialize the training and validation sets of images (train_ds and val_ds, respectively) by using a 80/20 train to test split. The class names are derived by the names of the folders the images exist in. 

The total number of training pictures divided by the batch size defined how many training runs the model will go through per epoch. Each run takes a batch size number of pictures to train on per run within the epoch. We used 32 for our training which would result in 3 runs of training per epoch.

The autotune variable ensures that the images are cached, speeding up the training. 

In [ ]:
batch_size = 32
img_height = 256
img_width = 256


train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split= 0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

class_names = train_ds.class_names
num_classes = len(class_names)
print(class_names)

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


Found 99 files belonging to 10 classes.
Using 80 files for training.
Found 99 files belonging to 10 classes.
Using 19 files for validation.
['1', '10', '2', '3', '4', '5', '6', '7', '8', '9']


# Build Neural Network

The neural network is a sequential model made up of many layers of resclaing, 2D convolutional networks, max pooling layers, dense layers. The convolutional networks utilizes 128 filters to maximize the accuracy while not requiring too much GPU usage. They also utilize a relu activation function. 

In [ ]:
model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(128, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(128, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(128, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(4096, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])


# Compile Neural Network

We compile the neural network by defining its optimizer function as the adam optimizer, a combination of two gradient descent methodologies, and a loss function of Sparse Categorial Cross entropy (detailed here: https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy). 


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


# Train the model

Fit the model with 200 epochs defining the training and validation set. (Epochs can be increased for better accuracy results)

In [ ]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=200
)

Epoch 1/200
3/3 [==============================] - 64s 7s/step - loss: 40.6876 - accuracy: 0.1625 - val_loss: 3.8565 - val_accuracy: 0.1579
Epoch 2/200
3/3 [==============================] - 2s 561ms/step - loss: 3.8952 - accuracy: 0.1125 - val_loss: 3.8375 - val_accuracy: 0.1053
Epoch 3/200
3/3 [==============================] - 2s 561ms/step - loss: 2.5711 - accuracy: 0.2750 - val_loss: 2.0479 - val_accuracy: 0.1053
Epoch 4/200
3/3 [==============================] - 2s 567ms/step - loss: 1.8838 - accuracy: 0.3750 - val_loss: 2.6446 - val_accuracy: 0.1053
Epoch 5/200
3/3 [==============================] - 2s 562ms/step - loss: 1.9433 - accuracy: 0.3750 - val_loss: 2.0307 - val_accuracy: 0.1053
Epoch 6/200
3/3 [==============================] - 2s 563ms/step - loss: 1.7267 - accuracy: 0.3750 - val_loss: 2.2079 - val_accuracy: 0.1053
Epoch 7/200
3/3 [==============================] - 2s 564ms/step - loss: 1.6839 - accuracy: 0.3750 - val_loss: 2.2401 - val_accuracy: 0.1053
Epoch 8/200
3/

# Check Accuracy

Evaluate the model and output simplified results. 

In [ ]:
test_loss, test_acc = model.evaluate(val_ds, verbose=2)

print('\nTest accuracy:', test_acc)


1/1 - 0s - loss: 5.6272 - accuracy: 0.4737 - 143ms/epoch - 143ms/step

Test accuracy: 0.4736842215061188


test the model